In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


   # Import data


In [2]:
data_affil = pd.read_pickle(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/first_clean/data_scholar_all_year_merge.pkl')[
        ['UT (Unique WOS ID)','Publication Year','Affiliations']];
data_affil['Affiliations2'] = data_affil['Affiliations']
data_affil = data_affil.rename(columns={"UT (Unique WOS ID)": "wos_ID"}).set_index(['wos_ID','Publication Year','Affiliations2'])

data_affil.head()

,,,Affiliations
wos_ID,Publication Year,Affiliations2,
wos:a1975ak27200035,1975,harvard university; harvard medical school,harvard university; harvard medical school
wos:a1975bb78600005,1975,max planck society,max planck society
wos:a1975ak49300025,1975,stanford university; stanford university,stanford university; stanford university
wos:a1975al49000001,1975,state university of new york (suny) system; state university of new york (suny) stony brook; suny maritime college; united states department of energy (doe); brookhaven national laboratory,state university of new york (suny) system; st...
wos:a1975ay53300001,1975,university of washington; university of washington seattle,university of washington; university of washin...


# data preprocessing

In [3]:
# split and expand the Affiliations columns
data_affil_expand = data_affil['Affiliations'].str.split("; ", expand=True).astype(np.uint8,errors='ignore')
data_affil_expand.head()

,,,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
wos_ID,Publication Year,Affiliations2,,,,,,,,,,,,,,,,,,,,,
wos:a1975ak27200035,1975,harvard university; harvard medical school,harvard university,harvard medical school,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
wos:a1975bb78600005,1975,max planck society,max planck society,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
wos:a1975ak49300025,1975,stanford university; stanford university,stanford university,stanford university,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
wos:a1975al49000001,1975,state university of new york (suny) system; state university of new york (suny) stony brook; suny maritime college; united states department of energy (doe); brookhaven national laboratory,state university of new york (suny) system,state university of new york (suny) stony brook,suny maritime college,united states department of energy (doe),brookhaven national laboratory,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
wos:a1975ay53300001,1975,university of washington; university of washington seattle,university of washington,university of washington seattle,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [4]:
import pandas as pd
import numpy as np
from functools import reduce

# Partition the data into smaller dataframes
partition_size = 1000
partitions = [data_affil_expand[i:i+partition_size] for i in range(0, len(data_affil_expand), partition_size)]

# Process each partition
processed_partitions = []
for partition in partitions:
    dfs = [pd.get_dummies(partition[col]) for col in partition]
    d = reduce(lambda x, y: x.add(y, fill_value=0), dfs)
    cols = [c for c in d.columns if c[-1:] != '.']
    df = d[cols]
    cols = [c for c in df.columns.to_list() if len(c) != 1]
    df = df[cols]

    # Replace 0.0 with np.nan
    df = df.replace(0.0, np.nan)

    # Replace non-zero values with 1
    df = df.where(df.isnull(), 1)
    #df = df.replace(to_replace=df[df != 0], value=1)
    
    processed_partitions.append(df)
    

# Concatenate all the processed partitions
result = pd.concat(processed_partitions)

result.head()

,,,aarhus university,academia sinica - taiwan,aix-marseille universite,ajinomoto co inc,albert einstein college of medicine,"all-russia research institute of animal physiology, biochemistry & nutrition",argonne national laboratory,atomic energy of canada limited,augusta university,australian national university,...,"rajiv gandhi university, itanagar",seokyeong university,songshan lake materials laboratory,universidad de la costa,university of balochistan,university of ilorin,university of southern queensland,va pittsburgh healthcare system,veterans affairs medical center - birmingham,xiamen university of technology
wos_ID,Publication Year,Affiliations2,,,,,,,,,,,,,,,,,,,,,
wos:a1975ak27200035,1975,harvard university; harvard medical school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wos:a1975bb78600005,1975,max planck society,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wos:a1975ak49300025,1975,stanford university; stanford university,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wos:a1975al49000001,1975,state university of new york (suny) system; state university of new york (suny) stony brook; suny maritime college; united states department of energy (doe); brookhaven national laboratory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wos:a1975ay53300001,1975,university of washington; university of washington seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#save data
result.to_csv(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/institutions_all.csv')
result.to_pickle(f'D:\PROJECT/YORK UNIVERSITY/BACTERIOPHAGES/data_analyse/institutions_all.pkl')

# institutions

## count

In [7]:
institutions_list = result.reset_index().set_index(['wos_ID','Publication Year']).count().to_frame().sort_values(by=0, ascending=False)
institutions_list['%'] = institutions_list[0]/institutions_list.T['Affiliations2'][0]*100
# #save data
# countries_list.to_csv(f'../TABLES/countries_list_split.csv'

institutions_list.head(60)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\dmbog\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dmbog\AppData\Local\Temp\ipykernel_11260\660303290.py", line 1, in <cell line: 1>
    institutions_list = result.reset_index().set_index(['wos_ID','Publication Year']).count().to_frame().sort_values(by=0, ascending=False)
  File "C:\Users\dmbog\anaconda3\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\dmbog\anaconda3\lib\site-packages\pandas\core\frame.py", line 5787, in reset_index
    new_obj = self.copy()
  File "C:\Users\dmbog\anaconda3\lib\site-packages\pandas\core\generic.py", line 6032, in copy
    data = self._mgr.copy(deep=deep)
  File "C:\Users\dmbog\anaconda3\lib\site-packages\pandas\core\internals\managers.py", line 603, in copy
    res = self.apply("copy", deep=deep)
  File "C:\Users\dmbog\ana